In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score

IMAGE_DIM = 28*28*1
NUM_CLASS = 10

# Ex 2-1

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

device = torch.device("cuda")

In [3]:
class SimpleNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        outputs = self.fc2(x)

        return outputs

In [4]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])
dataset1 = datasets.MNIST("", train=True, download=True, transform=transform)
dataset2 = datasets.MNIST("", train=False, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset1, batch_size=1024)
test_loader = torch.utils.data.DataLoader(dataset2, batch_size=1024)

model = SimpleNet(IMAGE_DIM, 64, NUM_CLASS).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

train_loss = []
train_acc = []
test_loss = []
test_acc = []
for n in range(30):
  model.train()

  train_loss_step = []
  train_acc_step = []
  for i, (x, y) in enumerate(train_loader):
    optimizer.zero_grad()
    x = x.reshape(x.shape[0], -1)
    x = x.to(device)
    y = y.to(device)

    pred = model(x)
    loss = criterion(pred, y)
    loss.backward()
    optimizer.step()

    train_loss_step.append(float(loss))
    acc = accuracy_score(y.to('cpu').detach().numpy().copy(), 
                          pred.argmax(dim=1).to('cpu').detach().numpy().copy())
    train_acc_step.append(acc)
  train_loss.append(np.mean(train_loss_step))
  train_acc.append(np.mean(train_acc_step))

  with torch.no_grad():
    model.eval()
    test_loss_step = []
    test_acc_step = []
    for i ,(x, y) in enumerate(test_loader):
      x = x.reshape(x.shape[0], -1)
      x = x.to(device)
      y = y.to(device)

      pred = model(x)
      loss = criterion(pred, y)

      test_loss_step.append(float(loss))
      acc = accuracy_score(y.to('cpu').detach().numpy().copy(), 
                         pred.argmax(dim=1).to('cpu').detach().numpy().copy())
      test_acc_step.append(acc)
  
  test_loss.append(np.mean(test_loss_step))
  test_acc.append(np.mean(test_acc_step))
  print("%i: %f, %f, %f, %f" % (n, train_acc[-1], test_acc[-1], train_loss[-1], test_loss[-1]))


0: 0.500591, 0.730096, 1.896067, 1.439043
1: 0.768745, 0.813371, 1.158920, 0.902423
2: 0.825954, 0.849468, 0.799085, 0.676955
3: 0.851640, 0.865519, 0.637903, 0.565581
4: 0.865385, 0.878003, 0.550915, 0.500131
5: 0.874539, 0.884275, 0.496734, 0.457124
6: 0.880267, 0.890458, 0.459709, 0.426680
7: 0.885013, 0.894063, 0.432741, 0.403974
8: 0.889096, 0.896407, 0.412160, 0.386344
9: 0.892021, 0.899884, 0.395865, 0.372201
10: 0.894260, 0.902356, 0.382566, 0.360551
11: 0.896715, 0.904602, 0.371438, 0.350739
12: 0.898961, 0.906224, 0.361936, 0.342322
13: 0.900898, 0.907854, 0.353670, 0.334981
14: 0.902465, 0.909154, 0.346366, 0.328497
15: 0.903921, 0.910326, 0.339840, 0.322687
16: 0.905422, 0.911693, 0.333936, 0.317448
17: 0.906730, 0.913285, 0.328540, 0.312663
18: 0.907834, 0.914096, 0.323564, 0.308267
19: 0.908982, 0.915133, 0.318945, 0.304196
20: 0.909892, 0.915711, 0.314627, 0.300409
21: 0.911001, 0.916169, 0.310570, 0.296861
22: 0.912044, 0.917048, 0.306734, 0.293521
23: 0.913181, 0.91795

# Ex 2-2

In [5]:
import keras
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.optimizers import SGD

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], -1)
y_train = y_train.reshape(y_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)
y_test = y_test.reshape(y_test.shape[0], -1)
x_train = x_train/255
x_test = x_test/255
x_train = (x_train-0.1307)/0.3081
x_test = (x_test-0.1307)/0.3081
y_train = to_categorical(y_train, NUM_CLASS)
y_test =  to_categorical(y_test, NUM_CLASS)

In [7]:
inputs = Input(shape=(IMAGE_DIM))
x = Dense(64, activation='relu')(inputs)
outputs = Dense(10, activation='softmax')(x)
model = Model(inputs=inputs, outputs=outputs)

model.compile(optimizer=SGD(learning_rate=0.01),
               loss='categorical_crossentropy',
               metrics=['accuracy'])

hist = model.fit(x_train, y_train,
                batch_size=1024,
                epochs=30,
                verbose=1,
                validation_data=(x_test, y_test))

Epoch 1/30
59/59 [==============================] - 0s 7ms/step - loss: 1.3633 - accuracy: 0.5831 - val_loss: 0.8182 - val_accuracy: 0.7763
Epoch 2/30
59/59 [==============================] - 0s 5ms/step - loss: 0.7096 - accuracy: 0.8016 - val_loss: 0.5746 - val_accuracy: 0.8446
Epoch 3/30
59/59 [==============================] - 0s 5ms/step - loss: 0.5525 - accuracy: 0.8455 - val_loss: 0.4792 - val_accuracy: 0.8652
Epoch 4/30
59/59 [==============================] - 0s 5ms/step - loss: 0.4791 - accuracy: 0.8647 - val_loss: 0.4277 - val_accuracy: 0.8770
Epoch 5/30
59/59 [==============================] - 0s 5ms/step - loss: 0.4353 - accuracy: 0.8758 - val_loss: 0.3948 - val_accuracy: 0.8875
Epoch 6/30
59/59 [==============================] - 0s 5ms/step - loss: 0.4054 - accuracy: 0.8837 - val_loss: 0.3717 - val_accuracy: 0.8932
Epoch 7/30
59/59 [==============================] - 0s 5ms/step - loss: 0.3833 - accuracy: 0.8899 - val_loss: 0.3544 - val_accuracy: 0.8976
Epoch 8/30
59/59 [==

In [8]:
model.evaluate(x_test, y_test, verbose=0)

[0.23550193011760712, 0.9315999746322632]